In [1]:
# https://arxiv.org/pdf/quant-ph/0104030.pdf
# ^^^ Need to be able to prepare arbitrary state!
import numpy as np
import cirq
from functools import reduce

In [ ]:
# https://arxiv.org/pdf/quant-ph/9503016.pdf

bottom of pg 663[https://rdo.psu.ac.th/sjstweb/journal/27-3/18mathices.pdf]

## Roots of diagonalizable matrices

In this section, we consider an nth root of a diagonalizable matrix.

- Theorem 2.1: Let A be an $m\times m$ complex matrix. If A is diagonalizable, then A has an nth root, for anypositive integer n.

Proof:

Let $A$ be a diagonalizable matrix, i.e., there exists a non-singular matrix S such that $A = SDS^{-1}$where $D=[d_{ij}]_{m\times m}$ is a diagonal matrix.

Let $D^{\frac{1}{n}}=[d_{ij}^{\frac{1}{n}}]_{m \times m}$, where $d_{ij}^{\frac{1}{n}}$ is an n-th root of $d_{ij}$.

So $A = S (D^{\frac{1}{n}})^{n} S^{-1} = (SD^{\frac{1}{n}}S)^{n}$ Therefore an n-th root of A exists.

https://math.stackexchange.com/questions/1168438/the-nth-root-of-the-2x2-square-matrix

In [251]:
from numpy import linalg as LA
class V_gate():
    
    # V^{n} = U
    
    def __init__(self,U, n):
        self.U=U
        self.n = n
        
        self.D = None
        self.V = None
        self.V_dag = None
        
    def _diagonalise_U(self):

        val,vec = np.linalg.eig(self.U)
        
        #sorting
        idx = val.argsort()[::-1]   
        val_sorted = val[idx]
        vec_sorted = vec[:,idx]
        
        # find diagonal matrix:
        vec_sorted_inv = np.linalg.inv(vec_sorted) 
        self.D = vec_sorted_inv.dot(mat.dot(vec_sorted))
        
        self.S=vec_sorted
        self.S_inv = vec_sorted_inv
        # where U = S D S^{-1}
        
        if not np.allclose(self.S.dot(self.D).dot(self.S_inv), self.U):
            raise ValueError('U != SDS-1')
    
    def Get_V_gate_matrices(self):
        
        if self.D is None:
            self._diagonalise_U()
        
        D_nth_root = np.power(self.D, 1/self.n)
#         D_nth_root = np.sqrt(self.D)
        
        self.V = self.S.dot(D_nth_root).dot(self.S_inv)
        self.V_dag = self.V.conj().transpose()
        
        if not np.allclose(reduce(np.matmul, [self.V for _ in range(self.n)]), self.U, atol=1e-3):
            raise ValueError('U != V^2')       
        
        return self.V, self.V_dag 
        
mat = cirq.X._unitary_()
aa = V_gate(mat, 2)
V, V_dag = aa.Get_V_gate_matrices()

np.around(V.dot(V), 3)

array([[ 0.+0.j,  1.+0.j],
       [ 1.-0.j, -0.-0.j]])

In [252]:
aa = V_gate(mat, 4)
V, V_dag = aa.Get_V_gate_matrices()
np.around(((V.dot(V)).dot(V)).dot(V), 3)

array([[ 0.+0.j,  1.+0.j],
       [ 1.-0.j, -0.-0.j]])

In [335]:
def int_to_Gray(num, n_qubits):
    # https://en.wikipedia.org/wiki/Gray_code
    
    # print(np.binary_repr(num)) # standard binary form!
    
    # The operator >> is shift right. The operator ^ is exclusive or
    gray_int = num^(num>>1)
    
    return np.binary_repr(gray_int,n_qubits)

for i in range(2**3):
    print(int_to_Gray(i, 3))
    
int_to_Gray(6, 4)

000
001
011
010
110
111
101
100


'0101'

In [334]:
def check_binary_str_parity(binary_str):
    """
    Returns 0 for EVEN parity
    Returns 1 for ODD parity    
    """
    parity = sum(map(int,binary_str))%2
    
    return parity

check_binary_str_parity('0101')

0

In [ ]:
# NOTE pg 17 of Elementary gates for quantum information
class n_control_U(cirq.Gate):
    """
    """

    def __init__(self, V, list_of_control_qubits, list_control_vals, U_qubit):
        self.V = V
        self.list_of_control_qubits = list_of_control_qubits
        self.list_control_vals = list_control_vals
        self.sys_qubit = sys_qubit
        
        self.n_ancilla=len(list_of_control_qubits)
        
    def flip_control_to_zero(self):
        for index, control_qubit in enumerate(self.list_control_qubits):
            if self.list_control_vals==0:
                yield cirq.X.on(control_qubit)
            

    def _decompose_(self, qubits):

        for control_val in range(1, 2**self.n_ancilla):
            gray_control_str = int_to_Gray(control_val, self.n_ancilla)
            
            if check_binary_str_parity(gray_control_str)==0:
                # even parity
                yield self.V.controlled(num_controls=1, control_values=[1]).on(U_qubit)
            else:
                # odd parity
                yield self.V_dagg.controlled(num_controls=1, control_values=[1]).on(U_qubit)
        
    def _circuit_diagram_info_(self, args):
        string_list = []
        for _ in range(self.num_qubits()):
            string_list.append('Changing to Z basis circuit')
        return string_list

    def num_qubits(self):
        return len(self.list_control_qubits) + len(self.list_system_qubits)

In [318]:
list(range(1,2**3))

[1, 2, 3, 4, 5, 6, 7]

1

In [324]:
sum(map(int,bits[1:]))%2

1

In [205]:

class n_qubit_Toffolli_gate(cirq.Gate):
    """
    https://arxiv.org/pdf/0708.3274.pdf#cite.Vatan
    
    https://arxiv.org/pdf/quant-ph/9503016.pdf
    pg 19!
    """

    def __init__(self, list_control_qubits, list_control_vals, list_system_qubits):

        self.list_control_qubits = list_control_qubits[::-1]
        self.list_control_vals= list_control_vals[::-1]
        self.list_system_qubits=list_system_qubits[::-1]
        
        self.V = ((1j+1)/2)*np.array([
                            [1, -1j],
                            [-1j, 1]
                            ])
        
    def flip_control_to_zero(self):
        for index, control_qubit in enumerate(self.list_control_qubits):
            if self.list_control_vals==0:
                yield cirq.X.on(control_qubit)

    def _decompose_(self, qubits):

        for i, system_qubit in enumerate(self.list_system_qubits):
            yield cirq.TOFFOLI(self.list_control_qubits[i], self.list_system_qubits[i+1], system_qubit)
        
    def _circuit_diagram_info_(self, args):
        string_list = []
        for _ in range(self.num_qubits()):
            string_list.append('Changing to Z basis circuit')
        return string_list

    def num_qubits(self):
        return len(self.list_control_qubits) + len(self.list_system_qubits)

In [206]:
list_control_qubits=cirq.LineQubit.range(1,6)
system_qubits = cirq.LineQubit.range(5,10)

test = n_qubit_Toffolli_gate(list_control_qubits, [], system_qubits)
circuit = test = cirq.Circuit(cirq.decompose_once(
        (test(*cirq.LineQubit.range(test.num_qubits())))))

IndexError: list index out of range

In [ ]:
    measure_PauliString_in_Z_basis = change_pauliword_to_Z_basis_then_measure(PauliWord_QubitOp)
    measure_PauliString_in_Z_basis_Q_circ = cirq.Circuit(cirq.decompose_once(
        (measure_PauliString_in_Z_basis(*cirq.LineQubit.range(measure_PauliString_in_Z_basis.num_qubits())))))

In [196]:
cirq.Circuit(cirq.TOFFOLI(cirq.LineQubit(1), cirq.LineQubit(2),cirq.LineQubit(3)))

1: ───@───
      │
2: ───@───
      │
3: ───X───

AttributeError: 'list' object has no attribute 'dimension'

In [213]:
cirq.X.controlled(num_controls=2, control_values=[0,1]).on(
                            *list(cirq.LineQubit.range(3)))

cirq.ControlledOperation(controls=(cirq.LineQubit(0), cirq.LineQubit(1)), sub_operation=cirq.X.on(cirq.LineQubit(2)), control_values=((0,), (1,)))

In [215]:
cirq.ControlledGate(cirq.X(1))

AttributeError: 'int' object has no attribute 'dimension'

In [208]:
qb = [cirq.LineQubit(i) for i in range(6)]

cnX = cirq.X.controlled_by(qb[0], qb[1], qb[2], qb[3], qb[4])

circuit = cirq.Circuit()
circuit.append(cnX(qb[5]))
circuit

AttributeError: '_PauliX' object has no attribute 'controlled_by'

In [224]:
op = cirq.X.controlled(num_controls=3, control_values=[0, 0, 1]).on(*[cirq.LineQubit(1), cirq.LineQubit(2), cirq.LineQubit(3)], cirq.LineQubit(4))
print(cirq.Circuit(op))


1: ───(0)───
      │
2: ───(0)───
      │
3: ───@─────
      │
4: ───X─────
